# Benchmark tutorial

This notebook provides basic tutorial of benchmark of bearing fault diagnosis model and optimizer's hyperparameter. Core implementation of this code is in the `fdob` module. This modlue provides data download, data preprocessing, model implementation, quasi-random hyperparameter sampling, and model trainning.

In [1]:
import fdob
import fdob.processing as processing
import fdob.model as model
import info
import benchmark
import sys, os
import torch
from torchvision import transforms

from vit_cl import pruning

/data/home/akdakd55/.pyenv/versions/3.9.6/envs/cl/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: attempted relative import with no known parent package

# Data download

We can download the CWRU and MFPT datasets using `download_cwru` and `download_mfpt`, respectively. These functions automatically download each dataset from URLs and return pandas `DataFrame`. `split_dataframe` splits dataframe to train, validation, and test `Dataframe`. `build_from_dataframe` build `numpy.ndarray` dataset by overlapping. In this tutorial, we use the CWRU dataset for training, and the data is generated with the sample length 4,096 and shift size 2,048.

In [2]:
df = fdob.download_cwru("./data/cwru")

# We exclude label named 999 and 0 HP motor load condition.
df = df[(df["label"] != 999) & (df["load"] != 0)]

train_df, val_df, test_df = fdob.split_dataframe(df, 0.6, 0.2)

X_train, y_train = fdob.build_from_dataframe(train_df, 4096, 2048, False)
X_val, y_val = fdob.build_from_dataframe(val_df, 4096, 2048, False)
X_test, y_test = fdob.build_from_dataframe(test_df, 4096, 2048, False)

# Getting the model and preparing `DataLoader`

The models and the initial hyperparameter search spaces for each optimizer used in the paper are in the `info.py` file. `info.model` contains the model, input length of the model, and `transform` of data. `info.hparam` contains the information of the search space of four optimizers, sgd, momentum, RMSProp, and adam. Users can employ the models and hyperparameter search space in the `info.py` but also can use the custom models and search space.

To train the model using PyTorch Lightning, `DataLoader` should be prepared. We provide `DatasetHandler`, which is the collection of multi-domain datasets. `assign` method generates `DataLoader`, and users can access the `DataLoader` by key of `DataLoader`. This tutorial uses two `DataLoader`s. `DataLoader` with key `cwru` is the noise-free data from the CWRU dataset, and `DataLoader` with key `cwru0` is the noisy data generated by Gaussian noise SNR 0dB from the CWRU dataset.

In [3]:
model_name = "transformer"

model = info.model[model_name]["model"]
sample_length = info.model[model_name]["sample_length"]
tf_data = info.model[model_name]["tf"]
tf_label = [processing.NpToTensor()]
batch_size = 32
num_workers = 1

dmodule = fdob.DatasetHandler()

dmodule.assign(
    X_train,
    y_train,
    X_val,
    y_val,
    X_test,
    y_test,
    sample_length,
    "cwru",
    transforms.Compose(tf_data),
    transforms.Compose(tf_label),
    batch_size,
    num_workers
)

dmodule.assign(
    X_train,
    y_train,
    X_val,
    y_val,
    X_test,
    y_test,
    sample_length,
    "cwru0",
    transforms.Compose([processing.AWGN(0)] + tf_data),
    transforms.Compose(tf_label),
    batch_size,
    num_workers
)

In [4]:
# access to the DataLoader of train dataset of the noise-free CWRU dataset.
dmodule.dataloaders["cwru"]["train"]

# Access to the Dataset of train dataset of the noisy CWRU dataset.
dmodule.dataloaders["cwru"]["train"].dataset

# Hyperparameter sampling

`log_qsample` samples hyperaprameters in log-scale quasi-random distribution. For example, if adam optimizer and hyperparameter search space defined in `info.py` is used, each hyperparameters are sampled from probability distributions below.

* $\eta \sim 10^{U[-4, -1]}$
* $1 - \beta_{1} \sim 10^{U[-3, 0]}$
* $1 - \beta_{2} \sim 10^{U[-4, -1]}$
* $\epsilon \sim 10^{U[-10, 0]}$

In [5]:
n_exps = 4

hparam_info = info.hparam["adam"]

hparams = fdob.log_qsample(
    hparam_info["n_params"],
    hparam_info["param_names"],
    hparam_info["lb"],
    hparam_info["ub"],
    hparam_info["reversed"],
    n_exps
)

hparams

{'lr': array([0.0001    , 0.00316228, 0.00056234, 0.01778279]),
 'beta1': array([0.999     , 0.99      , 0.9       , 0.99784557]),
 'beta2': array([0.9999    , 0.99960189, 0.99841511, 0.99369043]),
 'eps': array([1.00000000e-10, 2.68269580e-09, 7.19685673e-08, 1.93069773e-06])}

# Model training and evaluation

To benchmark, following materials should be prepared. 

* train `DataLoader`
* validation `DataLoader`
* PyTorch model
* model's keyword argument (if there is no keyword argument, pass `None`)
* PyTorch optimizer from `torch.optim`
* optimizer's keyword argument (if there is no keyword argument, pass `None`)
* PyTorch loss function from `torch.nn`
* loss function's keyword argument (if there is no keyword argument, pass `None`)
* The number of epochs
* Random seed (if `None` is passed, random seed is not set)
* The number of GPU (only CUDA GPU is supported)
* Result directory of the experiemnt

Following code train the WDCNN using the first hyperparameter determined above, and the result is saved in the `./logs/mytest`.


In [6]:
train_loader = dmodule.dataloaders["cwru"]["train"]
val_loader = dmodule.dataloaders["cwru"]["val"]

seed = 7777
n_gpu = 0
n_epochs = 100

In [11]:
model_kwargs = {
    "n_classes": 10,
    "embed_dim": 32,
    "depth": 4,
    "n_heads": 2,
    "attn_drop": 0.5,
    "mlp_drop": 0.5
}
model_train = model(**model_kwargs)

target_sparsity = None

prune_conv_only = False

callbacks = list()
if target_sparsity is not None:
    convs = [(module, 'weight') for module in model_train.modules() if isinstance(module, (torch.nn.Conv1d, torch.nn.Conv2d))]

    params_to_prune = convs
    if not prune_conv_only:
        fcs = [(module, 'weight') for module in model_train.modules() if isinstance(module, torch.nn.Linear)][:-1]
        params_to_prune = params_to_prune + fcs

    pruner = L1UnstructuredPruner()
    pruner.register(params_to_prune)

    sparsity_scheduler = DefaultSparsityScheduler(final_sparsity=target_sparsity, pruning_steps=n_epochs-1)

    pruning_callback = IterativePruningCallback(pruner=pruner, sparsity_scheduler=sparsity_scheduler, frequency=len(train_loader))

In [12]:
sparsity_str = ''.join(f'{target_sparsity:.2f}'.split('.')) if target_sparsity else 'noprune'
result_dir = f"./logs/{model_name}_{sparsity_str}"
result_dir = result_dir + '_conv_only' if prune_conv_only else result_dir

In [9]:
opt = hparam_info["optimizer"]
opt_kwargs = {
    "lr": hparams["lr"][0],
    "betas": (hparams["beta1"][0], hparams["beta2"][0]),
    "eps": hparams["eps"][0]
}

loss = torch.nn.CrossEntropyLoss
loss_kwargs = None

benchmark.train(
    train_loader,
    val_loader,
    model_train,
    model_kwargs,
    opt,
    opt_kwargs,
    loss,
    loss_kwargs,
    n_epochs,
    seed,
    n_gpu,
    result_dir,
    callbacks=callbacks
)
benchmark.test(
    dmodule.dataloaders["cwru"]["test"],
    model_train,
    model_kwargs,
    opt,
    opt_kwargs,
    loss,
    loss_kwargs,
    n_epochs,
    seed,
    n_gpu,
    result_dir,
    "noise-free"
)
benchmark.test(
    dmodule.dataloaders["cwru0"]["test"],
    model,
    model_kwargs,
    opt,
    opt_kwargs,
    loss,
    loss_kwargs,
    n_epochs,
    seed,
    n_gpu,
    result_dir,
    "noise"
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                | Params
------------------------------------------------
0 | model   | BaselineTransformer | 91.4 K
1 | loss_fn | CrossEntropyLoss    | 0     
------------------------------------------------
91.4 K    Trainable params
0         Non-trainable params
91.4 K    Total params
0.365     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/minha/.pyenv/versions/3.9.6/envs/dfb/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


/home/minha/.pyenv/versions/3.9.6/envs/dfb/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/minha/.pyenv/versions/3.9.6/envs/dfb/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1892: PossibleUserWarning: The number of training batches (42) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 99: 100%|██████████| 55/55 [00:01<00:00, 45.28it/s, loss=0.00362, v_num=0, val_loss=0.174, val_acc=0.995, train_loss=0.0078, train_acc=0.998] 

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 13/13 [00:00<00:00, 112.14it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.9877750873565674
        test_loss           0.17345502972602844
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 13/13 [00:00<00:00, 108.31it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.5403422713279724
        test_loss           1.3887505531311035
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 1.3887505531311035, 'test_acc': 0.5403422713279724}]

In [10]:
for seed in [12134234, 25235, 23423, 1212121, 123123]:

    benchmark.test(
        dmodule.dataloaders["cwru0"]["test"],
        model,
        model_kwargs,
        opt,
        opt_kwargs,
        loss,
        loss_kwargs,
        n_epochs,
        seed,
        n_gpu,
        result_dir,
        "noise"
    )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/minha/.pyenv/versions/3.9.6/envs/dfb/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 13/13 [00:00<00:00, 118.61it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.47677260637283325
        test_loss           1.4235447645187378
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 13/13 [00:00<00:00, 115.46it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.47921761870384216
        test_loss           1.4352469444274902
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 13/13 [00:00<00:00, 122.63it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.5012224912643433
        test_loss           1.4341174364089966
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 13/13 [00:00<00:00, 125.14it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.47677260637283325
        test_loss           1.4177396297454834
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 13/13 [00:00<00:00, 133.13it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.5036674737930298
        test_loss           1.3770586252212524
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [13]:
for seed in [12134234, 25235, 23423, 1212121, 123123]:

    benchmark.test(
        dmodule.dataloaders["cwru0"]["test"],
        model,
        model_kwargs,
        opt,
        opt_kwargs,
        loss,
        loss_kwargs,
        n_epochs,
        seed,
        n_gpu,
        result_dir,
        "noise"
    )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/minha/.pyenv/versions/3.9.6/envs/dfb/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 13/13 [00:00<00:00, 111.72it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.6112469434738159
        test_loss           1.2903497219085693
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 13/13 [00:00<00:00, 118.52it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.6185818910598755
        test_loss           1.2987806797027588
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 13/13 [00:00<00:00, 116.38it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.6088019609451294
        test_loss           1.3002971410751343
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 13/13 [00:00<00:00, 119.36it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc             0.621026873588562
        test_loss           1.3395968675613403
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 13/13 [00:00<00:00, 110.56it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc             0.645476758480072
        test_loss           1.2419430017471313
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
